# Data Processing

## Import modules

In [1]:
# import SparkSession
from pyspark.sql import SparkSession

In [2]:
import pyspark.sql.functions as fn
from pyspark.sql.types import StringType,DoubleType,IntegerType

## Set spark session

In [3]:
spark.sparkContext.appName

'PySparkShell'

## Load data

In [4]:
# Load csv Dataset 
df=spark.read.csv('data/sample_data.csv',inferSchema=True,header=True)
df.createOrReplaceTempView("dfTable")

22/05/10 17:47:33 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/10 17:47:48 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/10 17:48:03 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
22/05/10 17:48:18 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


## Inspect data

In [5]:
# columns of dataframe
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [6]:
# shape of dataset
df.count(),len(df.columns)

(33, 5)

In [7]:
# print dataframe schema
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [8]:
# display fisrt few rows of dataframe
df.show()
#df.show(10)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
|      4| 27|       9.0|     0|   Oppo|
|      5| 37|      23.0|     5|   Vivo|
|      5| 37|      23.0|     5|Samsung|
|      3| 22|       2.5|     0|  Apple|
|      3| 27|       6.0|     0|     MI|
|      2| 27|       6.0|     2|   Oppo|
|      5| 27|       6.0|     2|Samsung|
|      3| 37|      16.5|     5|  Apple|
|      5| 27|       6.0|     0|     MI|
|      4| 22|       6.0|     1|   Oppo|
|      4| 37|       9.0|     2|Samsung|
|      4| 27|       6.0|     1|  Apple|
|      1| 37|      23.0|     5|     MI|
|      2| 42|      23.0|     2|   Oppo|
|      4| 37|       6.0|     0|   Vivo|
+-------+---+----------+------+-------+
only showing top 20 rows



## Grouping data

In [9]:
# group by one column
df.groupBy('mobile').count().show(5)

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
|  Apple|    7|
+-------+-----+



In [10]:
# sort value counts
df.groupBy('mobile').count().orderBy('count',ascending=False).show(5)

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|  Apple|    7|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
+-------+-----+



In [11]:
# calculate statistical measures
df.groupBy('mobile').mean().show(5)

+-------+------------------+------------------+------------------+------------------+
| mobile|      avg(ratings)|          avg(age)|   avg(experience)|       avg(family)|
+-------+------------------+------------------+------------------+------------------+
|     MI|               3.5|            30.125|           10.1875|             1.375|
|   Oppo| 2.857142857142857|28.428571428571427|10.357142857142858|1.4285714285714286|
|Samsung| 4.166666666666667|28.666666666666668| 8.666666666666666|1.8333333333333333|
|   Vivo|               4.2|              36.0|              11.4|               1.8|
|  Apple|3.4285714285714284|30.571428571428573|              11.0|2.7142857142857144|
+-------+------------------+------------------+------------------+------------------+



In [12]:
# calculate statistical measures
df.groupBy('mobile').sum().show()

+-------+------------+--------+---------------+-----------+
| mobile|sum(ratings)|sum(age)|sum(experience)|sum(family)|
+-------+------------+--------+---------------+-----------+
|     MI|          28|     241|           81.5|         11|
|   Oppo|          20|     199|           72.5|         10|
|Samsung|          25|     172|           52.0|         11|
|   Vivo|          21|     180|           57.0|          9|
|  Apple|          24|     214|           77.0|         19|
+-------+------------+--------+---------------+-----------+



In [13]:
# calculate statistical measures
df.groupBy('mobile').max().show()

+-------+------------+--------+---------------+-----------+
| mobile|max(ratings)|max(age)|max(experience)|max(family)|
+-------+------------+--------+---------------+-----------+
|     MI|           5|      42|           23.0|          5|
|   Oppo|           4|      42|           23.0|          2|
|Samsung|           5|      37|           23.0|          5|
|   Vivo|           5|      37|           23.0|          5|
|  Apple|           4|      37|           16.5|          5|
+-------+------------+--------+---------------+-----------+



In [14]:
# calculate statistical measures
df.groupBy('mobile').min().show()

+-------+------------+--------+---------------+-----------+
| mobile|min(ratings)|min(age)|min(experience)|min(family)|
+-------+------------+--------+---------------+-----------+
|     MI|           1|      27|            2.5|          0|
|   Oppo|           2|      22|            6.0|          0|
|Samsung|           2|      22|            2.5|          0|
|   Vivo|           3|      32|            6.0|          0|
|  Apple|           3|      22|            2.5|          0|
+-------+------------+--------+---------------+-----------+



In [15]:
# use spark sql
spark.sql('''select mobile, count(*) as count from dfTable
        group by mobile''').show()

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
|  Apple|    7|
+-------+-----+



In [16]:
# use spark sql
spark.sql('''select mobile, min(experience), min(age) from dfTable
        group by mobile''').show()

+-------+---------------+--------+
| mobile|min(experience)|min(age)|
+-------+---------------+--------+
|     MI|            2.5|      27|
|   Oppo|            6.0|      22|
|Samsung|            2.5|      22|
|   Vivo|            6.0|      32|
|  Apple|            2.5|      22|
+-------+---------------+--------+



In [17]:
# Aggregation
df.groupBy('mobile').agg({'experience':'sum'}).show()

+-------+---------------+
| mobile|sum(experience)|
+-------+---------------+
|     MI|           81.5|
|   Oppo|           72.5|
|Samsung|           52.0|
|   Vivo|           57.0|
|  Apple|           77.0|
+-------+---------------+



## Load and Inspect data

In [18]:
# Load csv Dataset
rtdf=spark.read.csv('data/online_retail_dataset.csv',inferSchema=True,header=True)
rtdf.createOrReplaceTempView("rtTable")

In [19]:
# columns of dataframe
rtdf.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [20]:
# shape of dataset
rtdf.count(),len(rtdf.columns)

(541909, 8)

In [21]:
# print dataframe schema
rtdf.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [22]:
# display fisrt few rows of dataframe
rtdf.show()
#rtdf.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [23]:
# counting
rtdf.select(fn.count('StockCode')).show() 

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [24]:
# distinct count
rtdf.select(fn.countDistinct('StockCode')).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [25]:
# get minimun and maximun
rtdf.select(fn.min("Quantity"), fn.max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [26]:
# Variance and Standard Deviation
rtdf.select(fn.var_pop('Quantity'), fn.var_samp('Quantity'),
        fn.stddev_pop('Quantity'), fn.stddev_samp('Quantity')).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660883| 47559.39140929852|  218.08095663447745|   218.08115785023364|
+-----------------+------------------+--------------------+---------------------+



In [27]:
# use sql
spark.sql('''select count(StockCode) from rtTable''').show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [28]:
# Covariance and Correlation
rtdf.select(fn.corr('InvoiceNo', 'Quantity'), fn.covar_samp('InvoiceNo', 'Quantity'),
        fn.covar_pop('InvoiceNo', 'Quantity')).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085646314E-4|             1052.7280543926051|             1052.726077876501|
+-------------------------+-------------------------------+------------------------------+



In [29]:
# count with groupby
rtdf.groupBy("InvoiceNo", "CustomerId").count().show(5)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   545165|     16339|   20|
|   545289|     14732|   30|
|   545853|     16517|   17|
|   546236|     12370|   48|
|   546640|     14092|   13|
+---------+----------+-----+
only showing top 5 rows



In [30]:
# agg function
rtdf.groupBy('InvoiceNo').agg({'Quantity':'count'}).show(5)

+---------+---------------+
|InvoiceNo|count(Quantity)|
+---------+---------------+
|   536596|              6|
|   536938|             14|
|   537252|              1|
|   537691|             20|
|   538041|              1|
+---------+---------------+
only showing top 5 rows



In [31]:
# agg function
rtdf.groupBy('InvoiceNo').agg(fn.count('Quantity').alias('quan'),
        fn.expr('count(Quantity)')).show(5)

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   545583|  16|             16|
|  C546174|   1|              1|
|   547122|  16|             16|
|   547557|   1|              1|
|   548542|   8|              8|
+---------+----+---------------+
only showing top 5 rows



In [32]:
# agg function
rtdf.groupBy('InvoiceNo').agg({'Quantity':'min', 'UnitPrice':'max'}).show(5)

+---------+--------------+-------------+
|InvoiceNo|max(UnitPrice)|min(Quantity)|
+---------+--------------+-------------+
|  C540850|          1.25|           -1|
|   540976|          18.0|            2|
|   541432|          4.95|            3|
|   541518|         14.95|            2|
|   541783|         16.95|            1|
+---------+--------------+-------------+
only showing top 5 rows



In [33]:
# agg function
rtdf.groupBy('InvoiceNo').agg(fn.max('Quantity'),
        fn.min('Quantity')).show(5)

+---------+-------------+-------------+
|InvoiceNo|max(Quantity)|min(Quantity)|
+---------+-------------+-------------+
|  C540850|           -1|           -1|
|   540976|           24|            2|
|   541432|           30|            3|
|   541518|          120|            2|
|   541783|           36|            1|
+---------+-------------+-------------+
only showing top 5 rows



## UDF

In [34]:
# UDF
from pyspark.sql.functions import udf

In [35]:
# normal function 
def price_range(brand):
    if brand in ['Samsung','Apple']:
        return 'High Price'
    elif brand =='MI':
        return 'Mid Price'
    else:
        return 'Low Price'

In [36]:
# create udf using python function
brand_udf=udf(price_range,StringType())

In [37]:
# apply udf on dataframe
df.withColumn('price_range',brand_udf(df['mobile'])).show(10,False)

+-------+---+----------+------+-------+-----------+
|ratings|age|experience|family|mobile |price_range|
+-------+---+----------+------+-------+-----------+
|3      |32 |9.0       |3     |Vivo   |Low Price  |
|3      |27 |13.0      |3     |Apple  |High Price |
|4      |22 |2.5       |0     |Samsung|High Price |
|4      |37 |16.5      |4     |Apple  |High Price |
|5      |27 |9.0       |1     |MI     |Mid Price  |
|4      |27 |9.0       |0     |Oppo   |Low Price  |
|5      |37 |23.0      |5     |Vivo   |Low Price  |
|5      |37 |23.0      |5     |Samsung|High Price |
|3      |22 |2.5       |0     |Apple  |High Price |
|3      |27 |6.0       |0     |MI     |Mid Price  |
+-------+---+----------+------+-------+-----------+
only showing top 10 rows



In [38]:
# using lambda function
age_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())

In [39]:
# apply udf on dataframe
df.withColumn("age_group", age_udf(df.age)).show(10)

+-------+---+----------+------+-------+---------+
|ratings|age|experience|family| mobile|age_group|
+-------+---+----------+------+-------+---------+
|      3| 32|       9.0|     3|   Vivo|   senior|
|      3| 27|      13.0|     3|  Apple|    young|
|      4| 22|       2.5|     0|Samsung|    young|
|      4| 37|      16.5|     4|  Apple|   senior|
|      5| 27|       9.0|     1|     MI|    young|
|      4| 27|       9.0|     0|   Oppo|    young|
|      5| 37|      23.0|     5|   Vivo|   senior|
|      5| 37|      23.0|     5|Samsung|   senior|
|      3| 22|       2.5|     0|  Apple|    young|
|      3| 27|       6.0|     0|     MI|    young|
+-------+---+----------+------+-------+---------+
only showing top 10 rows



## Saving file

In [40]:
# save the dataframe as single csv 
df.coalesce(1).write.csv('data/df_data.csv', header='True', mode='overwrite')

In [41]:
# save the data into parquet format 
rtdf.write.parquet('data/retail_dataset_parquet', mode='overwrite')

In [42]:
# read the data from parquet format 
rtdf2=spark.read.parquet('data/retail_dataset_parquet')

In [43]:
rtdf2.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   554511|    23170|REGENCY TEA PLATE...|       1|5/24/2011 15:52|     3.29|      null|United Kingdom|
|   554511|    23171|REGENCY TEA PLATE...|       6|5/24/2011 15:52|     3.29|      null|United Kingdom|
|   554511|    23172|REGENCY TEA PLATE...|       2|5/24/2011 15:52|     3.29|      null|United Kingdom|
|   554511|    23173|REGENCY TEAPOT RO...|       1|5/24/2011 15:52|    19.96|      null|United Kingdom|
|   554511|    23174|REGENCY SUGAR BOW...|       2|5/24/2011 15:52|     8.29|      null|United Kingdom|
|   554511|    23175|REGENCY MILK JUG ...|       1|5/24/2011 15:52|     7.46|      null|United Kingdom|
|   554511|    23182|TOILET SIGN OCCUP...|       1|5/24/2011 15:

## Outlier

In [44]:
import numpy as np

In [45]:
wdf = spark.read.csv('data/winequality_white.csv',sep=';',inferSchema=True,header=True)

In [46]:
# columns of dataframe
wdf.columns

['fixed_acidity',
 'volatile_acidity',
 'citric_acid',
 'residual_sugar',
 'chlorides',
 'free_sulfur_ dioxide',
 'total_sulfur_dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [47]:
# shape of dataset
wdf.count(),len(wdf.columns)

(4898, 12)

In [48]:
# print dataframe schema
wdf.printSchema()

root
 |-- fixed_acidity: double (nullable = true)
 |-- volatile_acidity: double (nullable = true)
 |-- citric_acid: double (nullable = true)
 |-- residual_sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free_sulfur_ dioxide: double (nullable = true)
 |-- total_sulfur_dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)



In [49]:
# display fisrt few rows of dataframe
#wdf.show()
wdf.show(10)

+-------------+----------------+-----------+--------------+---------+--------------------+--------------------+-------+----+---------+-------+-------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_ dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+--------------------+--------------------+-------+----+---------+-------+-------+
|          7.0|            0.27|       0.36|          20.7|    0.045|                45.0|               170.0|  1.001| 3.0|     0.45|    8.8|      6|
|          6.3|             0.3|       0.34|           1.6|    0.049|                14.0|               132.0|  0.994| 3.3|     0.49|    9.5|      6|
|          8.1|            0.28|        0.4|           6.9|     0.05|                30.0|                97.0| 0.9951|3.26|     0.44|   10.1|      6|
|          7.2|            0.23|       0.32|           8.5|    0.058|                47.0|    

In [50]:
wdf.select('pH','sulphates','chlorides').summary().show()

+-------+-------------------+-------------------+--------------------+
|summary|                 pH|          sulphates|           chlorides|
+-------+-------------------+-------------------+--------------------+
|  count|               4898|               4898|                4898|
|   mean| 3.1882666394446693| 0.4898468762760325|  0.0457723560636995|
| stddev|0.15100059961506673|0.11412583394883222|0.021847968093728805|
|    min|               2.72|               0.22|               0.009|
|    25%|               3.09|               0.41|               0.036|
|    50%|               3.18|               0.47|               0.043|
|    75%|               3.28|               0.55|                0.05|
|    max|               3.82|               1.08|               0.346|
+-------+-------------------+-------------------+--------------------+



In [51]:
bounds = {
    c: dict(
        zip(["q1", "q3"], wdf.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in ['pH']
}

In [52]:
bounds

{'pH': {'q1': 3.09, 'q3': 3.28}}

In [53]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)

In [54]:
bounds

{'pH': {'q1': 3.09,
  'q3': 3.28,
  'lower': 2.8049999999999997,
  'upper': 3.5649999999999995}}

In [55]:
wdf2 = wdf.select(
    '*',
    *[
        fn.when(
            fn.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in ['pH']
    ]
)

In [56]:
wdf2.select('pH', 'pH_out').show(10)

+----+------+
|  pH|pH_out|
+----+------+
| 3.0|     0|
| 3.3|     0|
|3.26|     0|
|3.19|     0|
|3.19|     0|
|3.26|     0|
|3.18|     0|
| 3.0|     0|
| 3.3|     0|
|3.22|     0|
+----+------+
only showing top 10 rows



In [57]:
wdf2.select('pH','pH_out').filter('pH_out==1').show(10)

+----+------+
|  pH|pH_out|
+----+------+
|3.69|     1|
|3.63|     1|
|3.72|     1|
|3.61|     1|
|3.64|     1|
|3.64|     1|
|3.72|     1|
|3.72|     1|
|3.58|     1|
|3.58|     1|
+----+------+
only showing top 10 rows



In [58]:
wdf2.select('pH','pH_out').where('pH_out == 1').count()

75

In [59]:
wdf2.createOrReplaceTempView("wdf2Table")

In [60]:
spark.sql('''select pH,pH_out from wdf2Table where pH_out=1 limit 10''').show()

+----+------+
|  pH|pH_out|
+----+------+
|3.69|     1|
|3.63|     1|
|3.72|     1|
|3.61|     1|
|3.64|     1|
|3.64|     1|
|3.72|     1|
|3.72|     1|
|3.58|     1|
|3.58|     1|
+----+------+



In [61]:
# save the data into parquet format 
wdf2.write.csv('data/wdf2', header='True', mode='overwrite')

In [62]:
# Load csv Dataset
wdf3=spark.read.csv('data/wdf2',inferSchema=True,header=True)
wdf3.createOrReplaceTempView("wdf4Table")

In [63]:
wdf3.filter('pH_out==1').show(15)

+-------------+----------------+-----------+--------------+---------+--------------------+--------------------+-------+----+---------+-------+-------+------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_ dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|pH_out|
+-------------+----------------+-----------+--------------+---------+--------------------+--------------------+-------+----+---------+-------+-------+------+
|          6.0|            0.27|       0.28|           4.8|    0.063|                31.0|               201.0| 0.9964|3.69|     0.71|   10.0|      5|     1|
|          5.5|           0.485|        0.0|           1.5|    0.065|                 8.0|               103.0|  0.994|3.63|      0.4|    9.7|      4|     1|
|          5.9|            0.21|       0.28|           4.6|    0.053|                40.0|               199.0| 0.9964|3.72|      0.7|   10.0|      4|     1|
|          6.0|             0.1|       0.24|        